<a href="https://colab.research.google.com/github/andrewli4938/TrafficLightDetection/blob/main/Group1_Traffic_Light_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install ultralytics

In [6]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import glob
import shutil
from PIL import Image
from google.colab import drive

from ultralytics import YOLO

In [7]:
# import our dataset
import kagglehub
source_path = kagglehub.dataset_download("mbornoe/lisa-traffic-light-dataset")

100%|██████████| 4.21G/4.21G [00:20<00:00, 218MB/s]

Extracting model files...


In [8]:
%pwd
%rm -r data
%mkdir data  # root folder for data

%cd data
%mkdir images
%mkdir labels

%cd images
%mkdir train
%mkdir val
%mkdir test

%cd ../labels
%mkdir train
%mkdir val
%mkdir test

%cd /content

rm: cannot remove 'data': No such file or directory
/content/data
/content/data/images
/content/data/labels
/content


In [9]:
# these two only have total 1000 images, and have duplicate filenames so we can deal with them later
%mv {source_path}/sample-dayClip6 {source_path}/..
%mv {source_path}/sample-nightClip1 {source_path}/..

In [10]:
# move all of the filenames into list
image_paths = glob.glob(f"{source_path}/**/*.jpg", recursive=True)

# process the annotations
target = "frameAnnotationsBOX.csv"
annotations = pd.DataFrame()
annotation_paths = glob.glob(f"{source_path}/**/{target}", recursive=True)
for p in annotation_paths:
  # combine the csv files into one
  new_frame = pd.read_csv(p, sep=";")
  new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
  annotations = pd.concat([annotations, new_frame])
filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
annotations["Filename"] = filenames
annotations.set_index("Filename", inplace=True)
annotations

,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y
Filename,,,,,
daySequence2--00143.jpg,stop,1178,350,1193,372
daySequence2--00144.jpg,stop,1152,351,1167,378
daySequence2--00145.jpg,stop,1126,351,1141,378
daySequence2--00146.jpg,stop,1100,352,1115,370
daySequence2--00147.jpg,stop,1074,350,1092,372
...,...,...,...,...,...
nightSequence2--06532.jpg,go,962,252,994,312
nightSequence2--06532.jpg,go,716,350,723,362
nightSequence2--06532.jpg,stop,651,333,657,343


In [11]:
def move_files(paths_list, destination_dir):
  for file in paths_list:
    shutil.copy(file, destination_dir)

In [12]:
# TODO: split the data into train, val, test
image_train, image_val_test = train_test_split(image_paths, train_size=0.7, test_size=0.3, random_state=42, shuffle=True)
image_val, image_test = train_test_split(image_val_test, train_size=0.5, random_state=42, shuffle=True)

print(len(image_train))
print(len(image_val))
print(len(image_test))

image_train_dir = "/content/data/images/train"
image_val_dir = "/content/data/images/val"
image_test_dir = "/content/data/images/test"
move_files(image_train, image_train_dir)
move_files(image_val, image_val_dir)
move_files(image_test, image_test_dir)

print(len(os.listdir(image_train_dir)))
print(len(os.listdir(image_val_dir)))
print(len(os.listdir(image_test_dir)))

30111
6452
6453
30111
6452
6453


In [13]:
# import yaml file: defines image locations and encodes classes into numbers for the model
!wget https://raw.githubusercontent.com/andrewli4938/TrafficLightDetection/refs/heads/main/data.yaml

--2024-11-04 23:35:44--  https://raw.githubusercontent.com/andrewli4938/TrafficLightDetection/refs/heads/main/data.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177 [text/plain]
Saving to: ‘data.yaml’

data.yaml           100%[===================>]     177  --.-KB/s    in 0s      

2024-11-04 23:35:44 (8.12 MB/s) - ‘data.yaml’ saved [177/177]



In [14]:
def convert_to_xywh(box_coordinates, image_dimensions):
  image_width, image_height = image_dimensions
  x1, y1, x2, y2 = box_coordinates
  width, height = x2-x1, y2-y1
  x_center = (x1+x2)/2
  y_center = (y1+y2)/2

  x_center = x_center/image_width
  y_center = y_center/image_height
  width = width/image_width
  height = height/image_height

  return (x_center, y_center, width, height)

In [15]:
"""
The *.txt file should be formatted with one row per object in
class x_center y_center width height format. Box coordinates
must be in normalized xywh format (from 0 to 1). If your boxes
are in pixels, you should divide x_center and width by image width,
and y_center and height by image height. Class numbers should be
zero-indexed (start with 0).
"""
image_dims = (1280, 960)

def create_labels(source_dir, destination_dir, annotations_df, encoding):
  bounding_box_df = annotations_df.iloc[:, 1:]
  for file in os.listdir(source_dir):
    write_path = os.path.join(destination_dir, file.replace(".jpg", ".txt"))
    if file in annotations_df.index:
      rows = annotations_df.loc[file]
      if isinstance(rows, pd.Series):
        rows = pd.DataFrame([rows])
    else:
      continue
    with open(write_path, "w") as image_label:
      for i in range(len(rows)):  # iterate through each label for current image
        class_number = encoding[rows.iloc[i, 0]]
        bounding_box = bounding_box_df.iloc[i]
        # print(bounding_box)
        # print(f"image is: {file} at index {i}")
        # print(type(rows))
        xywh = convert_to_xywh(bounding_box, image_dims)  # calculate bounding box coordinates here
        image_label.write(f"{class_number} {xywh[0]} {xywh[1]} {xywh[2]} {xywh[3]}\n")


In [16]:
class_encoding = {"stop": 0, "stopLeft": 1, "warning": 2, "warningLeft": 3,
                  "warningLeft": 4, "go": 5, "goForward": 6, "goLeft": 6}

label_train_dir = "/content/data/labels/train"
label_val_dir = "/content/data/labels/val"
label_test_dir = "/content/data/labels/test"

create_labels(image_train_dir, label_train_dir, annotations, class_encoding)
create_labels(image_val_dir, label_val_dir, annotations, class_encoding)
create_labels(image_test_dir, label_test_dir, annotations, class_encoding)

print(len(os.listdir(label_train_dir)))
print(len(os.listdir(label_val_dir)))
print(len(os.listdir(label_test_dir)))

25473
5412
5380


In [24]:
torch.cuda.is_available()

True

In [22]:
# model training
model = YOLO("yolo11n.pt")
results = model.train(data="/content/data.yaml", epochs=8, imgsz=512, batch=32)

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data.yaml, epochs=8, time=None, patience=100, batch=32, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/data/labels/train.cache... 25473 images, 4638 backgrounds, 0 corrupt: 100%|██████████| 30111/30111 [00:00<?, ?it/s]
Exception ignored in: <function _ConnectionBase.__del__ at 0x7f2df8287370>
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/data/labels/val.cache... 5412 images, 1040 backgrounds, 0 corrupt: 100%|██████████| 6452/6452 [00:00<?, ?it/s]


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 4 dataloader workers
Logging results to runs/detect/train5
Starting training for 8 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       6452      16295       0.32      0.293      0.276      0.146

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        8/8      3.02G      2.276      1.479     0.8419        113        512: 100%|██████████| 941/941 [04:10<00:00,  3.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:31<00:00,  3.23it/s]


                   all       6452      16295      0.595      0.265      0.291      0.158

8 epochs completed in 0.631 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 5.4MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLO11n summary (fused): 238 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:36<00:00,  2.80it/s]


                   all       6452      16295      0.594      0.265      0.291      0.158
                  stop       2792       6704      0.567      0.428      0.461       0.24
              stopLeft       1562       1918      0.789      0.503      0.599      0.385
               warning        150        344          1          0     0.0215      0.005
                    go         39         57      0.397       0.14      0.121     0.0795
             goForward       2780       6903      0.481      0.425      0.433      0.173
                goLeft        276        369      0.331     0.0921       0.11     0.0647
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train5


In [23]:
results.results_dict

{'metrics/precision(B)': 0.5941866568173838,
 'metrics/recall(B)': 0.26474354129965144,
 'metrics/mAP50(B)': 0.2909734242726103,
 'metrics/mAP50-95(B)': 0.15782585534392155,
 'fitness': 0.17114061223679042}

In [ ]:
#TODO: the label files are producing duplicate lines, fix the create_labels() function

In [ ]:
"""
OVERVIEW
Basically our LISA dataset contains around 44k images and a bunch of labels

Labels are:
  - light status: {go, slow, stop}
  - bounding box of where the stoplight is xyxy coordinates (top left, bottom right)
  - each image may have many labels since there are many stoplights in a single image

DaySequence
"""

In [ ]:
"""
END HERE
"""

'\nEND HERE\n'

In [ ]:
# # centralize both annotations and images (not scattered in different folders)
# annotations = pd.DataFrame()

# target = "frameAnnotationsBOX.csv"
# annotation_paths = glob.glob(f"{path}/**/{target}", recursive=True)

# # I am going to omit the approximately 1000 pictures and labels in sample-dayClip6
# # and sample-nightClip1 because it is really annoying to work with right now and and
# # we have 43k other values we can work with first
# for p in annotation_paths:
#   tokens = p.split("/")
#   if tokens[-2] == "sample-nightClip1" or tokens[-2] == "sample-dayClip6":  # omit here
#     continue

#   # combine the csv files into one
#   new_frame = pd.read_csv(p, sep=";")
#   new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
#   annotations = pd.concat([annotations, new_frame])

#   # move all of the nested image files into /content/images_dir
#   if len(tokens)==14:
#     subpath = path+"/"+tokens[-3]+"/"+tokens[-3]+"/"+tokens[-2]+"/frames"
#   else:
#     subpath = path+"/"+tokens[-2]+"/"+tokens[-2]+"/frames"
#   !cp -r {subpath}/* /content/images_dir

# filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
# annotations["Filename"] = filenames
# annotations  # NOTE: our labels are in this df here

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y
0,nightSequence2--00000.jpg,stop,566,335,572,348
1,nightSequence2--00000.jpg,stop,695,308,705,329
2,nightSequence2--00000.jpg,stop,745,335,752,345
3,nightSequence2--00000.jpg,stop,678,331,684,341
4,nightSequence2--00001.jpg,stop,564,333,571,346
...,...,...,...,...,...,...
18509,nightSequence1--04860.jpg,go,873,86,1018,206
18510,nightSequence1--04861.jpg,go,946,74,1091,188
18511,nightSequence1--04862.jpg,go,1016,51,1161,177
18512,nightSequence1--04863.jpg,go,1085,42,1245,159


In [ ]:
# #TODO: define custom dataset class
# class TrafficLightDataset(Dataset):
#   def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):
#     self.img_labels = annotations_df
#     self.img_dir = img_dir
#     self.transform = transform
#     self.target_transform = target_transform

#   def __len__(self):
#     return len(self.img_labels)

#   def __getitem__(self, idx):
#     img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#     image = read_image(img_path)
#     label = tuple(self.img_labels.iloc[idx, 1:])
#     if self.transform:
#       image = self.transform(image)
#     if self.target_transform:
#       pass  # not sure how this will work on a tuple (worry about it later)
#     return image, label


In [ ]:
# #TODO: split our data into train and test dataloaders

# img_dir = "content/images_dir"
# dataset = TrafficLightDataset(annotations, img_dir, transform=None, target_transform=None)

# # shuffle our indices before splitting (need day and night in both sets)
# indices = torch.randperm(len(dataset))
# train_size = int(0.8*len(dataset))
# train_indices = indices[:train_size]
# test_indices = indices[train_size:]

# # split into train and test
# train_dataset = torch.utils.data.Subset(dataset, train_indices)
# test_dataset = torch.utils.data.Subset(dataset, test_indices)

# # we have our dataloaders here
# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=True)